In [ ]:
#                                                ::  Felicity  ::

In [ ]:
#

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, LSTM
from tensorflow.keras.optimizers import Adam


#----------------------------------------------start JUNO------------------------------------------------------------------ 

# Generator model
def build_generator(latent_dim, output_shape):
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.5))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.5))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(output_shape), activation='linear'))
    model.add(Reshape(output_shape))
    return model

# Discriminator model
def build_discriminator(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.8))
    model.add(Dense(1, activation='linear'))
    return model

# RNN model
def build_rnn(latent_dim, input_shape):
    model = Sequential()
    model.add(LSTM(2000, input_shape=input_shape, return_sequences=True))
    model.add(LSTM(2000, return_sequences=True))
    model.add(LSTM(2000))
    model.add(Dense(1, activation='linear'))
    return model

# GAN model
def build_gan(generator, discriminator, input_shape):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(Reshape((input_shape[0], input_shape[1])))
    model.add(discriminator)
    return model


#----------------------------------------------end JUNO------------------------------------------------------------------ 



# Potential and example data loading (replace with your own data loading logic)
file_path = r'C:\Users\Subhadeep Maishal\Music\JUP\test_data\CBN_INPUT.csv'
df = pd.read_csv(file_path, parse_dates=['DATE'])
features = df[['AAO', 'AO', 'DMI', 'NAO', 'ONI', 'PDO', 'SOI', 'TNA', 'TSA', 'WHWP', 'WP']]
target = df['MEIv2']
features = (features - features.min()) / (features.max() - features.min())
np.random.seed(1000)




# Build and compile the discriminator
discriminator = build_discriminator((features.shape[1], 1))
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.2, 0.5), metrics=['accuracy'])




# Build and compile the generator
latent_dim = 100
generator = build_generator(latent_dim, (features.shape[1], 1))
generator.compile(loss='binary_crossentropy', optimizer=Adam(0.2, 0.5))




# Build and compile the RNN
rnn = build_rnn(latent_dim, (features.shape[1], 1))
rnn.compile(loss='mean_squared_error', optimizer=Adam(0.2, 0.5))




# Build and compile the GAN with RNN
discriminator.trainable = False
gan_rnn = build_gan(generator, discriminator, (features.shape[1], 1))
gan_rnn.compile(loss='binary_crossentropy', optimizer=Adam(0.2, 0.5))





# Training GAN with RNN
epochs = 50
batch_size = 32

half_batch = int(batch_size / 2)

for epoch in range(epochs):
    idx = np.random.randint(0, features.shape[0], half_batch)
    real_data = features.iloc[idx].values  # Convert to numpy array

    noise = np.random.normal(0, 1, (half_batch, latent_dim))
    generated_data = generator.predict(noise)

    real_labels = np.ones((half_batch, 1))
    fake_labels = np.zeros((half_batch, 1))

    d_loss_real = discriminator.train_on_batch(real_data, real_labels)
    d_loss_fake = discriminator.train_on_batch(generated_data, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    valid_labels = np.ones((batch_size, 1))

    g_loss = gan_rnn.train_on_batch(noise, valid_labels)

    if epoch % 100 == 0:
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")


        
        
        
# Generate data using RNN >>
num_samples = 420
noise = np.random.normal(0, 1, (num_samples, latent_dim))
generated_data = generator.predict(noise)
synthetic_data = rnn.predict(generated_data.reshape((num_samples, features.shape[1], 1)))




# Plot actual and generated data
plt.figure(figsize=(12, 8))
plt.plot(df['DATE'], target, label='Actual', marker='o', color='black')
plt.plot(df['DATE'].iloc[-num_samples:], synthetic_data[:, 0], label='Generated', marker='o', color='blue')
plt.xlabel('Date', fontsize=14)
plt.ylabel('MEIv2 Values', fontsize=14)
plt.title('Actual vs Generated MEIv2 Values Over Time', fontsize=16)
plt.legend(fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()





In [ ]:
#                                       ::   Cheers!  ::

In [ ]:
#                                       ::  Subhadeep  ::